## MLflow's Model Registry

In [39]:
import mlflow
from mlflow.tracking import MlflowClient

# MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
MLFLOW_TRACKING_URI = "http://127.0.0.1:5000/"
mlflow.set_tracking_uri(uri=MLFLOW_TRACKING_URI)

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [40]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/281592431142832501', creation_time=1716803121678, experiment_id='281592431142832501', last_update_time=1716803121678, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/346357645414705188', creation_time=1715948676242, experiment_id='346357645414705188', last_update_time=1715948676242, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/901325434016866654', creation_time=1715948588628, experiment_id='901325434016866654', last_update_time=1715948588628, lifecycle_stage='active', name='MLflow Quickstart', tags={}>]

In [ ]:
client.create_experiment(name="my-cool-experiment")

Let's check the latest versions for the experiment with id `1`...

In [41]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='346357645414705188',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [42]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: d290e621d20b434b964634d7316c64da, rmse: 6.3084
run id: 4c82333e7ab347e58cfa0b7d233303c9, rmse: 6.3084
run id: 8ac3b6ef9b034fc5b87172c0cc719959, rmse: 6.3101
run id: cd82dda29ea94beeb549124e58b8226a, rmse: 6.3127
run id: 1a4db62e1bee4dbea6b80b23095505e4, rmse: 6.3161


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [ ]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [43]:
run_id = "d290e621d20b434b964634d7316c64da"
model_name = "nyc-taxi-xgboost"
model_uri = f"runs:/{run_id}/model"

# mlflow.register_model(model_uri=model_uri, name=model_name)
# create "champion" alias for version 1 of model "nyc-taxi-xgboost"
# client.set_registered_model_alias(model_name, "champion", 1)

In [46]:
import pprint


pprint.pp(client.get_registered_model(name=model_name))
print()
latest_versions = client.get_registered_model(name=model_name).latest_versions
pprint.pp(latest_versions)
print()
for version in latest_versions:
    print(f"version: {version.version}, aliases: {version.aliases} ") 



<RegisteredModel: aliases={'challenger': '2', 'champion': '1'}, creation_timestamp=1716803343654, description='', last_updated_timestamp=1716804973953, latest_versions=[<ModelVersion: aliases=['challenger'], creation_timestamp=1716804746661, current_stage='None', description='', last_updated_timestamp=1716804746661, name='nyc-taxi-xgboost', run_id='d290e621d20b434b964634d7316c64da', run_link='', source='mlflow-artifacts:/346357645414705188/d290e621d20b434b964634d7316c64da/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>], name='nyc-taxi-xgboost', tags={}>

[<ModelVersion: aliases=['challenger'], creation_timestamp=1716804746661, current_stage='None', description='', last_updated_timestamp=1716804746661, name='nyc-taxi-xgboost', run_id='d290e621d20b434b964634d7316c64da', run_link='', source='mlflow-artifacts:/346357645414705188/d290e621d20b434b964634d7316c64da/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>]

v

In [45]:
# get a model version by alias
client.get_model_version_by_alias(model_name, "champion")

<ModelVersion: aliases=['champion'], creation_timestamp=1716803343663, current_stage='None', description='', last_updated_timestamp=1716803343663, name='nyc-taxi-xgboost', run_id='d290e621d20b434b964634d7316c64da', run_link='', source='mlflow-artifacts:/346357645414705188/d290e621d20b434b964634d7316c64da/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [ ]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

In [ ]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [ ]:
df = read_dataframe("data/green_tripdata_2021-03.csv")

In [ ]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

In [ ]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [ ]:
X_test = preprocess(df, dv)

In [ ]:
target = "duration"
y_test = df[target].values

In [ ]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

In [ ]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)